In [ ]:
!pip install osmnx
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 1.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import folium
import random
from google.colab import files
from sklearn.datasets import make_blobs
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist

import osmnx as ox
import networkx as nx
from IPython.display import IFrame

In [ ]:
ox.config(log_console=True, use_cache=True)

G_walk = ox.graph_from_place('Arequipa, Peru',network_type='walk')



<ipython-input-3-b82447549162>:1: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(log_console=True, use_cache=True)


In [ ]:
coordenadas = [[-16.391947447156777, -71.53540671439708]
               ,[-16.39318257297588, -71.52227461888903]
               ,[-16.394088286080212, -71.52206002731346]
               ,[-16.441757971039852, -71.57342970927253]
               ,[-16.447602666505095, -71.58742011167652]
               ,[-16.448261212708413, -71.5911966620187]
               ,[-16.457727567394773, -71.57497466168523]
               ,[-16.412064509215774, -71.50551436681512]
               ,[-16.407567763332345, -71.49839131754405]
               ,[-16.407117491216923, -71.49643330495691]
               ,[-16.409971678067055, -71.49703202034192]]
print(coordenadas)

[[-16.391947447156777, -71.53540671439708], [-16.39318257297588, -71.52227461888903], [-16.394088286080212, -71.52206002731346], [-16.441757971039852, -71.57342970927253], [-16.447602666505095, -71.58742011167652], [-16.448261212708413, -71.5911966620187], [-16.457727567394773, -71.57497466168523], [-16.412064509215774, -71.50551436681512], [-16.407567763332345, -71.49839131754405], [-16.407117491216923, -71.49643330495691], [-16.409971678067055, -71.49703202034192]]


In [ ]:
def distancia_e(lat1, lng1, lat2, lng2):
        return np.sqrt((lat2 - lat1)**2 + (lng2 - lng1)**2)

def clusterParadas_km(casas, k, max_iters = 1000):
      num_samples = len(casas)
      # print(casas)
      centroids = random.sample(list(casas), k)
      asignaciones = []

      for punto in casas:
          distancia_minima = float('inf')
          indice_centroide = None
          # print (punto[0])
          for i, centroide in enumerate(centroids):
            distancia = distancia_e(punto[0],punto[1], centroide[0], centroide[1])
            if distancia < distancia_minima:
              distancia_minima = distancia
              indice_centroide = i
          asignaciones.append(indice_centroide)

      iteracion  = 0
      while iteracion < max_iters:
          nuevos_centroides = []
          for i in range (k):
              puntos_asignados = [punto for punto, asignacion in zip(casas, asignaciones) if asignacion == i]
              if puntos_asignados:
                  nuevos_centroides.append([sum(coordenada) / len(puntos_asignados) for coordenada in zip(*puntos_asignados)])
              else:
                  nuevos_centroides.append(centroids[i])
          for i in range (len(centroids)):
              if centroids[i][1] == nuevos_centroides[i][1] and centroids[i][0] == nuevos_centroides[i][0]:
                  break
          centroids = nuevos_centroides
          asignaciones = []

          for punto in casas:
            distancia_minima = float('inf')
            indice_centroide = None
            #print (punto[0])
            for i, centroide in enumerate(centroids):
              distancia = distancia_e(punto[0],punto[1], centroide[0], centroide[1])
              if distancia < distancia_minima:
                distancia_minima = distancia
                indice_centroide = i
            asignaciones.append(indice_centroide)

          iteracion += 1
      return centroids, asignaciones

In [ ]:
def fitness(centroids, data):
    distances = pairwise_distances(data, centroids)
    return np.sum(np.min(distances, axis=1))

def genetic_algorithm(initial_centroids, k, num_dimensions, population_size, num_generations, mutation_rate, data):
    population = [initial_centroids + np.random.randn(k, num_dimensions) * 0.1 for _ in range(population_size)]
    for generation in range(num_generations):
        fitness_values = [fitness(centroids, data) for centroids in population]

        parents_indices = np.argsort(fitness_values)[:population_size // 2]
        parents = [population[i] for i in parents_indices]

        offspring = []
        for _ in range(population_size - len(parents)):
            parent_indices = np.random.choice(range(len(parents)), size=2, replace=False)
            parent1, parent2 = parents[parent_indices[0]], parents[parent_indices[1]]
            crossover_point = np.random.randint(k)
            child = np.vstack((parent1[:crossover_point], parent2[crossover_point:]))
            if np.random.rand() < mutation_rate:
                mutation_index = np.random.randint(k)
                mutation_offset = np.random.randn(1, num_dimensions)
                child[mutation_index] += mutation_offset[0]
            offspring.append(child)

        population = parents + offspring

        average_fitness = np.mean(fitness_values)

    best_solution = population[np.argmin(fitness_values)]
    return best_solution


In [ ]:
def nearest_unvisited(point, unvisited_points):
    min_distance = float('inf')
    nearest_point = None
    lat1, lng1 = point
    for p in unvisited_points:
        lat2, lng2 = p
        dist = distancia_e(lat1, lng1, lat2, lng2)
        if dist < min_distance:
            min_distance = dist
            nearest_point = p
    return nearest_point

def order_points(points):
    ordered_points = []
    unvisited_points = points.copy()

    current_point = unvisited_points.pop(0)
    ordered_points.append(current_point)

    while unvisited_points:
        nearest_point = nearest_unvisited(current_point, unvisited_points)
        if nearest_point is not None:
            ordered_points.append(nearest_point)
            unvisited_points.remove(nearest_point)
            current_point = nearest_point
        else:
            # If no nearest unvisited point was found, restart the search from any unvisited point
            current_point = unvisited_points.pop(0)
            ordered_points.append(current_point)

    return ordered_points

In [ ]:
inicial_node = [-16.39924482094625, -71.55439671695771]

In [ ]:
class Clustering:
    def __init__(self, coordenadas, k):
        self.puntos = np.zeros(len(coordenadas))  # Matriz de coordenadas de paradas
        self.casas = np.zeros((len(coordenadas),2))   # Matriz de coordenadas de casas
        self.paradas = np.zeros(k)
        self.casasParadaTemp = np.zeros(len(coordenadas), dtype=int)

        for i, coord in enumerate(coordenadas):
            self.casas[i][0] = coord[0]
            self.casas[i][1] = coord[1]

        self.paradas, self.puntos = clusterParadas_km(self.casas, len(self.paradas))
        # self.paradas = ordenar_paradas(self.paradas)

    def paradasCorrectas(self, matrizDistancias):
        # Implementa tu lógica para verificar si las paradas son correctas
        # Devuelve True si las paradas son correctas, False si no lo son
        if(matrizDistancias > 500):
          return False
        return True
    def improve_precision(self, coordenadas):
        data_array_samples = np.array(coordenadas)
        sample_size = 11
        random_indices = np.random.choice(len(data_array_samples), size=sample_size, replace=False)
        data = data_array_samples[random_indices]

        num_dimensions = data.shape[1]

        population_size = 500
        num_generations = 100
        mutation_rate = 0.1

        population = [self.paradas + np.random.randn(len(self.paradas), num_dimensions) * 0.1 for _ in range(population_size)]

        self.paradas = genetic_algorithm(self.paradas, len(self.paradas),
                                         num_dimensions, population_size,
                                         num_generations=num_generations,
                                         mutation_rate = mutation_rate, data = data)

        # self.paradas = ordenar_paradas(self.paradas)


    # def clusterParadas(self):
    #     matrizDistancias = np.zeros((len(self.casas[0]), len(self.puntos[0])))
    #     print (matrizDistancias, "a")
    #     print (self.puntos, "b")
    #     puntosAux = np.copy(self.puntos)
    #     print (puntosAux, "a")
    #     flag = True

    #     for iteraciones in range(len(self.casas[0])):
    #         puntosAux[0] = np.copy(self.puntos[0])
    #         puntosAux[1] = np.copy(self.puntos[1])
    #         matrizDistancias = np.zeros((len(self.casas[0]), len(self.puntos[0])))

    #         for i in range(len(self.casas[0])):
    #             for j in range(i, len(self.puntos[0])):
    #                 matrizDistancias[i][j] = self.distancia(self.casas[0][i], self.casas[1][i], self.puntos[0][j], self.puntos[1][j])
    #                 matrizDistancias[j][i] = matrizDistancias[i][j]

    #         #if not self.paradasCorrectas(matrizDistancias):
    #             #return puntosAux

    #     return puntosAux


In [ ]:
k = 4
clustering = Clustering(coordenadas, k)
print("Puntos agrupados:" , clustering.puntos, clustering.paradas)
# clustering.improve_precision(coordenadas)
# print("Nuevos Puntos tras algoritmo geneticos", clustering.puntos, clustering.paradas)

Puntos agrupados: [0, 0, 0, 2, 2, 2, 2, 1, 3, 3, 3] [[-16.39307276873762, -71.52658045353319], [-16.412064509215774, -71.50551436681512], [-16.44883735441203, -71.58175528616326], [-16.408218977538777, -71.4972855476143]]


In [ ]:
m = folium.Map(location=[inicial_node[0], inicial_node[1]],
              zoom_start=45)

loc = [(inicial_node[0], inicial_node[1])]
folium.CircleMarker([inicial_node[0], inicial_node[1]],
                        radius=20,
                        weight=5).add_to(m)
folium.Marker([inicial_node[0], inicial_node[1]],
              popup = 'Lugar de recogida',
              icon = folium.Icon(color='green',icon='briefcase', prefix='fa')).add_to(m)

for i in clustering.paradas:
  loc.append(i)

loc = order_points(loc)
folium.PolyLine(loc,
                 color='red',
                 weight=4,
                 opacity=0.8).add_to(m)

j=0
for i in coordenadas:
    j = j+1
    folium.Marker(i,
                  popup = f'casa num {j}',
                  icon = folium.Icon(color='purple',icon='home', prefix='fa')).add_to(m)

for i in clustering.paradas:
    j = j+1
    folium.Marker(i,
                  popup = f'Parada num {j}',
                  icon = folium.Icon(color='orange',icon='bus-simple', prefix='fa')).add_to(m)
m

In [ ]:
orig_node = ox.nearest_nodes(G_walk, inicial_node[1],inicial_node[0])

In [ ]:
# print(clustering.paradas[0][0], clustering.paradas[0][1])
# print(orig_node)

# dest_node = ox.nearest_nodes(G_walk,clustering.paradas[0][1],clustering.paradas[0][0])
# ruta = nx.shortest_path(G_walk, orig_node,dest_node,weight='length')
# route_map = ox.plot_route_folium(G_walk, ruta)

# new_ruta = nx.shortest_path(G_walk, dest_node,dest_node,weight='length')
# print(ruta)
# print(new_ruta)
# for i in new_ruta:
#   ruta.append(i)
#   print(i)

# ruta.append(1)
# print(ruta)


In [ ]:
dest_nodes = []
for i in loc[1:]:
    dest_node = ox.nearest_nodes(G_walk, i[1], i[0])
    dest_nodes.append(dest_node)

# print(dest_nodes)
# print(len(dest_nodes))
# print(orig_node)
ruta = nx.shortest_path(G_walk, orig_node, dest_nodes[0], weight='length')
length = nx.shortest_path_length(G_walk, orig_node, dest_nodes[0], weight= 'length')
# print(length)
# print(ruta)
j = 0

for i in dest_nodes[1:]:
    ruta_aux = nx.shortest_path(G_walk, dest_nodes[j], i, weight='length')
    length_aux = nx.shortest_path_length(G_walk, dest_nodes[j], i, weight= 'length')
    print (dest_nodes[j], i)
    print(ruta_aux)
    j=j+1
    length = length + length_aux
    for k in ruta_aux[1:]:
        ruta.append(k)

ruta_aux = nx.shortest_path(G_walk, dest_nodes[-1], orig_node, weight='length')

for i in ruta_aux[1:]:
  ruta.append(i)
# print (ruta_aux)
# print(ruta)
# print(length)
route_map = ox.plot_route_folium(G_walk, ruta, route_map = m)
# route_map.save('route.html')

531270116 1264285479
[531270116, 5375739511, 5375739517, 5375739516, 1264319163, 1264368114, 4553183280, 1264299965, 1264367119, 1264288386, 1264285805, 5483337510, 5377989891, 5381265053, 1264340535, 5469339516, 5469339514, 11002043137, 5394910653, 11002043139, 11024162773, 5394909329, 11024162761, 5473715303, 1253003811, 1264339121, 11024160495, 11024160499, 1264290715, 11024162717, 1253016345, 7567877264, 1264303820, 1264300313, 5394254300, 2085228821, 5467966186, 5394268805, 5394268804, 5483314226, 4733211729, 5394271257, 11023828005, 4733211731, 5474144803, 5377955765, 8234719387, 5377955763, 8234719386, 5704995333, 1264288294, 5703156644, 5377955779, 5703156643, 5394271226, 5703156642, 5703156641, 5474144810, 5703156640, 1253002237, 5377955777, 1264300628, 5378003185, 1264340871, 1264285479]
1264285479 1264342449
[1264285479, 1264340871, 5378003185, 1264300628, 2085240745, 1264288649, 1264319708, 1264334748, 5072490762, 1264347600, 5415365208, 5415365206, 5394254223, 10062265077,

<ipython-input-22-e67b0ff42883>:32: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  route_map = ox.plot_route_folium(G_walk, ruta, route_map = m)


In [ ]:
route_map

In [ ]:
print("Peso del total del recorrido", length, "metros")

Peso del total del recorrido 19507.303 metros


Recorrido del grafo usando algoritmo geneticos

23556.97 metros

Usando algoritmos de clustering:

19507.303 metros